In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
from scipy import stats
from statsplots import one_d_plotter, two_d_plotter

In [2]:
interact(one_d_plotter, pop_mean = widgets.FloatSlider(min = -2, max = 2, step = 0.1),
        sample_mean = widgets.FloatSlider(min = -2, max = 2, step = 0.1),
        i = widgets.IntSlider(25, min = 0, max = 360, step = 1),
        j = widgets.IntSlider(50, min = 0, max = 360, step = 1));

interactive(children=(FloatSlider(value=0.0, description='pop_mean', max=2.0, min=-2.0), FloatSlider(value=0.0…

In [ ]:
def two_d_plotter(vmean, i, j):    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')

    y1 = np.random.normal(loc = vmean, scale = 1, size = 10_000)
    y2 = np.random.normal(loc = vmean+1, scale = 1, size = 10_000)
    y3 = np.random.normal(loc = vmean+2, scale = 1, size = 10_000)

    hist1, bins1 = np.histogram(y1, bins = 30)
    hist2, bins2 = np.histogram(y2, bins = 30)
    hist3, bins3 = np.histogram(y3, bins = 30)

    x1 = (bins1[:-1] + bins1[1:])/2
    x2 = (bins2[:-1] + bins2[1:])/2
    x3 = (bins3[:-1] + bins3[1:])/2
    
    ax.bar(x1, hist1, zs=0, zdir='y', color=c, ec=c, alpha=0.8)
    ax.bar(x2, hist2, zs = 1, zdir = 'y', color = c, ec = c, alpha = 0.8)
    ax.bar(x3, hist3, zs=2, zdir='y', color=c, ec=c, alpha=0.8)
#     ax.bar(x2, hist2, zs = 1, zdir = 'y')
#     ax.plot(np.zeros(len(x1)), np.ones(len(x1))*np.mean(x1), np.zeros(len(x2)), color = 'red')
    ax.set_ylim(-1, 1)
    ax._axis3don = False
    ax.view_init(i, j)
    #ax.set_title('Linear Regression Setting')
    plt.title('$\mu_y = b_0 + b_1 * x$')

In [ ]:
two_d_plotter(3, 25, 50)

### Example

Consider a linear model for decrease in blood pressure over a four-week period with $\mu_y = 2.8 + 0.8x$ and standard deviation $\sigma = 3.2$.  The explanatory variable $x$ is the number of servings of fruits and vegetables in a controlled diet.

### The Statistical Framing

$$y_i = \beta_0 + \beta_1x_i +  \epsilon_i$$

where $\epsilon$ reprent the errors and we have some assumptions about these:

- they are *iid*(independent and identically distributed)
- they are roughly normally distributed with mean 0
- the normal distribution has standard deviation $\sigma$

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [ ]:
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")

In [ ]:
df = duncan_prestige.data

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
#build a linear model with education as input and
#prestige as target


In [ ]:
#make predictions for each data point


### Residuals

The **residual** of our model is the error in our predictions.  Specifically, we have:

$$\begin{align}
e_i = \text{observed} - \text{predicted}  \\
  = y_i - \hat{y_i}  \\
 = y_i - (b_0 + b_1x_i)
 \end{align}$$
 
We use these residuals to understand the error in the model, specifically to determine $\sigma$ using the formula:

$$\begin{align}\displaystyle
s^2 = \frac{\sum e_i^2}{n - 2} \\
 = \frac{\sum (y_i - \hat{y_i})^2}{n-2}
 \end{align}$$
 
thus the **model standard deviation** $\sigma$ is given by 

$$\sigma = \sqrt{s^2}$$

In [ ]:
class StatLinReg(LinearRegression):
    pass

In [ ]:
lr = StatLinReg()

In [ ]:
X = df[['education']]
y = df['prestige']

In [ ]:
lr.fit(X, y)

In [ ]:
preds = lr.predict(X)

In [ ]:
residuals = [yi - yhat for yi, yhat in zip(y, preds)]

In [ ]:
plt.plot(X, residuals, 'o')
plt.axhline(color = 'red')
plt.title('Residual Plot', loc = 'left');

**PROBLEM**

Add to the `StatLinReg` class to include a residual method that accepts a feature matrix `X` and returns a numpy array of residuals.  

In [ ]:
class StatLinReg(LinearRegression):
    def residuals(self, X, y):
        resids = [yi - yhat for yi, yhat in zip(y, self.predict(X))]
        return np.array(resids)

In [ ]:
lr = StatLinReg()
lr.fit(X, y)
resids = lr.residual(X, y)

In [ ]:
plt.plot(X, resids, 'o')

**PROBLEM**

- Add another method that will return the standard deviation of the model called `std` using the formula above.

- Plot a histogram of the residuals, is this normal?

- Use seaborn's residual plot to generate a plot of the residuals and include a lowess line (check the arguments!)



<div class = "alert-warning">
  <center>
<strong>CHECK YOUR ASSUMPTIONS</strong>
 </center>
    
    
<li>Data is from a simple random sample of population</li>
    <li>There is a linear relationship present</li>
    <li>Standard deviation about the population line is the same for all values of explanatory variable.</li>
    <li>The response varies normally about the population regression line, i.e. the model deviations vary normally about 0.</li>
</div>

In [ ]:
#probability plot from scipy
stats.probplot(resids, plot = plt);

### Standard Errors for Regression Coefficients

- **Standard error of the slope**:

$$SE_{b_1} = \displaystyle\frac{s}{\sqrt {\sum (x_i - x)^2}}$$

- **Standard error of the intercept**:

$$SE_{b_0} = s \sqrt{\frac{1}{n} - \frac{\bar{x}^2}{\sum(x_i - \bar{x})^2}}$$

### Confidence Intervals and Significance

$$\text{estimate} \pm t*SE_{estimate}$$

where $t$ is our critical point from the $t$ distribution.




#### CL for slope

$$b_1 \pm t*SE_{b_1}$$


Note that the degrees of freedom for our $t$ distribution is $n-2$.

We can test the hypothesis $$H_0: \beta_1 = 0$$

by examining the **test statistic**
$$t = \frac{b_1}{SE_{b_1}}$$

**A SMALL EXAMPLE**

| umbilical cord diameter | gestational age |
| ----------------------   |  ------------  |
| 2 | 16 |
| 6 | 18 |
| 9 | 26 |
| 14 | 33 |
| 21 | 28 |
| 23 | 39 |

In [ ]:
df = pd.DataFrame({'umb_diameter': [2, 6, 9, 14, 21, 23],
                  'gestational_age': [16, 18, 26, 33, 28, 39]})

In [ ]:
df

In [ ]:
plt.scatter(df['umb_diameter'], df['gestational_age'])

In [ ]:
#examine the data
df

In [ ]:
X = df[['umb_diameter']]
y = df['gestational_age']

In [ ]:
#instantiate
lr = LinearRegression()
#fit
lr.fit(X, y)
#predict
preds = lr.predict(X)

In [ ]:
#look at the coefficients
lr.coef_

In [ ]:
#examine the intercept
lr.intercept_

In [ ]:
#compute the residuals
resids = [(yi - yhat) for yi, yhat in zip(y, preds)]

In [ ]:
#use residuals to determine standard deviation
s = np.sqrt(sum([r**2 for r in resids])/ (len(resids) - 2))

In [ ]:
#set mean of x to xbar
xbar = np.mean(X.values)

In [ ]:
#determine the standard error of slope
seb1 = s/np.sqrt(sum([(xi - xbar)**2 for xi in X.values]))

In [ ]:
#look at the standard error
seb1

In [ ]:
#determine the test statistic
slope = lr.coef_
tstat = slope/seb1
tstat

### Testing Two Sides

Whether or not the slope is non-zero!

In [ ]:
from scipy import stats

In [ ]:
2*(1 - stats.t(4).cdf(3.65))

In [ ]:
#95% confidence interval for slope
upper = (lr.coef_ + stats.t(4).interval(0.95)[1]*seb1)[0]
lower = (lr.coef_ + stats.t(4).interval(0.95)[0]*seb1)[0]

In [ ]:
(lower, upper)

*WHAT DOES IT ALL MEAN?!*


### BMI Example

In [ ]:
#read in data
import pandas as pd
bmi = pd.read_excel('datasets/PABMI.xls')

In [ ]:
#peek at head
bmi.head()

In [ ]:
#plot steps vs. bmi
plt.scatter(bmi['PA'], bmi['BMI'])

In [ ]:
#fit the model
lr = LinearRegression()
lr.fit(bmi[['PA']], bmi['BMI'])

In [ ]:
#examine coefs 
lr.coef_

In [ ]:
#examine intercept
lr.intercept_

In [ ]:
#standard error??


In [ ]:
lower = lr.coef_ + stats.t(98).interval(0.95)[1]*0.158

In [ ]:
upper = lr.coef_ - stats.t(98).interval(0.95)[1]*0.158

In [ ]:
lower, upper

### Testing for Correlation

$$t = \frac{r\sqrt{n - 2}}{\sqrt{1 - r^2}} $$

where $n$ is the sample size and $r$ is the sample correlation.  

In [ ]:
X =bmi['PA']
y = bmi['BMI']
#correlation between bmi and pa
np.corrcoef(X.values.ravel(), y)[0, 1]

In [ ]:
#compute the test statistic above; 
#have we seen this before?


In [ ]:
# What's this do? stats.pearsonr(X, y)
